In [ ]:
!pip install datasets

! pip install -U datasets[audio]
! pip install yt-dlp
! pip install gradio

!pip install librosa
! pip install -U datasets[audio]

In [ ]:
import pandas as pd
from tqdm import tqdm
import os
import torchvision.transforms as T
import torch
import librosa
import numpy as np
import scipy.io.wavfile as wav
import ntpath

from numpy.lib import stride_tricks
from matplotlib import pyplot as plt

import librosa
import IPython
from tqdm import tqdm
import numpy as np
import pandas as pd
import scipy
from torchvision import transforms, datasets
import subprocess
from pathlib import Path
from datasets import load_dataset



In [ ]:
ds = load_dataset('google/MusicCaps', split='train')


In [ ]:
def download_clip(
    video_identifier,
    output_filename,
    start_time,
    end_time,
    tmp_dir='/tmp/musiccaps',
    num_attempts=5,
    url_base='https://www.youtube.com/watch?v='
):
    status = False

    # Construct the command to download the video clip using yt-dlp
    command = f"""
        yt-dlp --quiet --no-warnings -x --audio-format wav -f bestaudio -o "{output_filename}" --download-sections "*{start_time}-{end_time}" {url_base}{video_identifier}
    """.strip()

    attempts = 0
    while True:
        try:
            output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            attempts += 1
            if attempts == num_attempts:
                return status, err.output
        else:
            break

    # Check if the video was successfully saved.
    status = os.path.exists(output_filename)
    return status, 'Downloaded'

from datasets import Audio
def process(example):
    outfile_path = str(data_dir / f"{example['ytid']}.wav")
    status = True
    if not os.path.exists(outfile_path):
        status = False
        status, log = download_clip(
            example['ytid'],
            outfile_path,
            example['start_s'],
            example['end_s'],
        )

    example['audio'] = outfile_path
    example['download_status'] = status
    return example

# ПОМЕНЯТЬ НА 5000
samples_to_load = 5000     # How many samples to load
cores = 4                 # How many processes to use for the loading
sampling_rate = 44100     # Sampling rate for the audio, keep in 44100
writer_batch_size = 1000  # How many examples to keep in memory per worker. Reduce if OOM.
data_dir = "./music_data" # Where to save the data

# Just select some samples 
ds = ds.select(range(samples_to_load))

# Create directory where data will be saved
data_dir = Path(data_dir)
data_dir.mkdir(exist_ok=True, parents=True)

ds = ds.map(
        process,
        num_proc=cores,
        writer_batch_size=writer_batch_size,
        keep_in_memory=False
    ).cast_column('audio', Audio(sampling_rate=sampling_rate))

Map (num_proc=4):   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import Dataset

class SongsDataloader(Dataset):

    def __init__(self, path_to_audio):
      super().__init__()

      directory = path_to_audio
      s = 0
      self.songs_data = np.zeros([5000,1,512,512])
      transform = T.Resize((512,512))
      to_delete = []
      for filename in tqdm(os.listdir(directory)):
        f = os.path.join(directory, filename)
        try: 
            if os.path.isfile(f):
                audio_data = f
                sw, sr = librosa.load(audio_data)
                X = librosa.stft(sw)
                Xdb = librosa.amplitude_to_db(abs(X))
                x = Xdb[None,:]
                t = torch.from_numpy(x)
                ss = transform(t)
                self.songs_data[s] = ss
        except:
            to_delete.append(s)
            s+=1
      print(to_delete)    
      self.describes = np.array(ds['caption'])
      self.len_ = self.songs_data.shape[0]


    def __len__(self):
      return self.len_

    def __getitem__(self, index):    
      x = self.describes[index]
      y = self.songs_data[index]


      return x, y

In [ ]:
z = SongsDataloader('music_data')

 16%|█▋        | 820/4973 [01:05<05:14, 13.22it/s]<ipython-input-9-ee29d0e21e42>:18: UserWarning: PySoundFile failed. Trying audioread instead.
  sw, sr = librosa.load(audio_data)
100%|██████████| 4973/4973 [06:18<00:00, 13.13it/s]

[0, 1, 2, 3]


In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(z, batch_size=16, shuffle=True)

In [ ]:
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
pipe.to("cuda")

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git
!pip install -U -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-x5td3fa9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-x5td3fa9
  Resolved https://github.com/huggingface/diffusers.git to commit d82b032319984bad3bf3a897f4826773dd422144
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 KB 12.6 MB/s eta 0:00:00
